In [1]:
import numpy
import os
import pandas
import tensorflow

from datetime import datetime
from sklearn.model_selection import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *

C:\Users\yamia\AppData\Local\Temp\ipykernel_9356\217575311.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
print("TensorFlow:", tensorflow.__version__)
print("GPU:", tensorflow.config.list_physical_devices('GPU'))

TensorFlow: 2.15.0
GPU: []


In [3]:
gpus = tensorflow.config.list_physical_devices('GPU')
if gpus:
    try:
        tensorflow.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tensorflow.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [2]:
df = pandas.read_csv('animal_chess.csv')

In [3]:
df = df[:100000]

In [4]:
count = len(df)

In [5]:
sample = df.iloc[32]
sample

board     ----r-E-Tl-----WC--d---------p------L--R--P---...
side                                                     -1
piece                                                     L
atk                                                       0
move                                                   F1E1
river                                                     0
trap                                                      1
den                                                       0
score                                                   -90
winner                                                    0
Name: 32, dtype: object

In [6]:
# Encode the piece
def encode_piece(piece_char):
    piece_mapping = {'-': 0, 'r': 1, 'c': 2, 'd': 3, 'w': 4, 'p': 5, 't': 6, 'l': 7, 'e': 8, 'R': -1, 'C': -2, 'D': -3, 'W': -4, 'P': -5, 'T': -6, 'L': -7, 'E': -8}
    return piece_mapping.get(piece_char, 0)

encode_piece(sample['piece'])

-7

In [7]:
# Encode the board
def encode_board(board_str):
    board_matrix = numpy.zeros((9, 7))
    for i, piece in enumerate(board_str[::-1]):
        row, col = divmod(i, 9)
        board_matrix[col][row] = encode_piece(piece)
    return numpy.flip(numpy.flip(board_matrix, 0), 1)

encode_board(sample['board'])

array([[ 0.,  7.,  0.,  0., -7.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  5.,  0.,  0.,  4.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-8., -4.,  0.,  0., -5.,  0.,  0.],
       [ 0., -2.,  0.,  0.,  0., -3.,  0.],
       [-6.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
df['board_encoded'] = df['board'].apply(encode_board)
board_matrix_flattened = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
board_matrix_flattened

array([[ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       ...,
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.]])

In [9]:
X = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
y = df['score'].values

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
import os
from datetime import datetime

# Định nghĩa hàm xây dựng mô hình
def build_model(input_shape, activation='relu'):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(2048, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])
    return model

# Tạo model mới với hàm kích hoạt là LeakyReLU
model = build_model((9, 7, 1), activation='leaky_relu')

# Log directory cho TensorBoard
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Chuẩn bị callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, save_format='tf')

# Huấn luyện mô hình
results = []
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    history = model.fit(X_train.reshape(-1, 9, 7, 1), y_train, epochs=50, batch_size=256,
                        validation_data=(X_test.reshape(-1, 9, 7, 1), y_test),
                        callbacks=[reduce_lr, early_stopping, checkpoint, tensorboard_callback])

    results.append(model.evaluate(X_test.reshape(-1, 9, 7, 1), y_test))

print("Fold results:", results)
print("Average result:", np.mean(results, axis=0))

Epoch 1/50
313/313 [==============================] - 9s 26ms/step - loss: 8657.2725 - mae: 38.9233 - val_loss: 6810.3110 - val_mae: 33.6431 - lr: 1.0000e-04
Epoch 2/50
313/313 [==============================] - 18s 58ms/step - loss: 3935.9917 - mae: 26.8483 - val_loss: 3118.7195 - val_mae: 24.1648 - lr: 1.0000e-04
Epoch 3/50
313/313 [==============================] - 9s 29ms/step - loss: 2109.3442 - mae: 22.9798 - val_loss: 2032.2216 - val_mae: 23.5548 - lr: 1.0000e-04
Epoch 4/50
313/313 [==============================] - 8s 25ms/step - loss: 1447.5656 - mae: 20.5345 - val_loss: 1590.6737 - val_mae: 23.0478 - lr: 1.0000e-04
Epoch 5/50
313/313 [==============================] - 8s 25ms/step - loss: 1140.1051 - mae: 18.5525 - val_loss: 1872.1396 - val_mae: 31.0715 - lr: 1.0000e-04
Epoch 6/50
313/313 [==============================] - 18s 56ms/step - loss: 966.5275 - mae: 17.2466 - val_loss: 1007.5986 - val_mae: 15.9031 - lr: 1.0000e-04
Epoch 7/50
313/313 [==============================]